In [ ]:
import os
import re
import sys
import math
import numpy as np
from numpy.linalg import inv
from scipy.signal import lsim
from pathlib import Path
from tqdm.notebook import tqdm
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
# from run_PF import compute_fourier_coeffs
from filter_OU_inputs import fit_TF, filter_inputs

In [ ]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
fontsize = 9
FIGURES_DIR = Path('figures')

In [ ]:
data_dir = Path('../data/Sardinia/default_config/GFM_01/Ta_4')
assert data_dir.exists(), f'{data_dir}: no such directory.'

Load the AC data: the data file contains the transfer functions computed by `compute_spectra.py`.

In [ ]:
AC_data_file = data_dir / 'V2020_Rete_Sardegna_2021_06_03cr_GFM_AC_TF_-6.0_4.0_50.npz'
assert AC_data_file.exists(), f'{AC_data_file}: no such file'
AC_data = np.load(AC_data_file, allow_pickle=True)
frequencies = AC_data['F']

Load the transient data: the data file contains the results of the simulations performed with PowerFactory.

In [ ]:
F = 10
tran_data_file = data_dir / f'V2020_Rete_Sardegna_2021_06_03cr_GFM_tran_pref_{F}_Hz.npz'
assert tran_data_file.exists(), f'{tran_data_file}: no such file'
blob = np.load(tran_data_file, allow_pickle=True)
tran_time = blob['time'].astype(float)
tran_data = blob['data'].item()
tran_config = blob['config'].item()
input_fun = np.vectorize(eval(tran_config['inputs'][0]['waveform']), otypes=[list])

In [ ]:
device_names = blob['device_names'].item()
device_types_dict = {'bus': 'ElmTerm', 'genstat': 'ElmGenstat'}
var_names_dict = {'bus': 'm:fe', 'genstat': 's:xspeed'}
devices_to_consider = 'genstat', 'bus'

##### Define the inputs and outputs among the variables available in the data file:

In [ ]:
def find_full_var_names(all_var_names, device_names, device_types, var_names):
    full_var_names = []
    for dev_name, dev_type in zip(device_names, device_types):
        for var_name in var_names:
            full_var_names += [
                name for name in all_var_names if
                    re.match(f'.*-{dev_name}\\.{dev_type}\\.{var_name}$', name) is not None
            ]
    return full_var_names

Inputs:

In [ ]:
input_names = ['Genstat_01']
all_input_names = AC_data['input_names'].tolist()
assert all(name in all_input_names for name in input_names), "At least one input name missing from data file"

Outputs:

In [ ]:
all_var_names = AC_data['var_names'].tolist()
full_var_names = []
max_N_vars = 2
for dev in devices_to_consider:
    tmp = find_full_var_names(
        all_var_names,
        device_names[dev][:max_N_vars],
        [device_types_dict[dev] for _ in range(max_N_vars)],
        [var_names_dict[dev].split(':')[1]]
    )
    assert len(tmp) == max_N_vars, "Could not find all requested variables"
    full_var_names += tmp
N_inputs = len(input_names)
N_vars = len(full_var_names)
input_idx = [all_input_names.index(name) for name in input_names]
var_idx = [all_var_names.index(name) for name in full_var_names]

Get the appropriate transfer functions:

In [ ]:
J, K = np.meshgrid(input_idx, var_idx, indexing='ij')
TF = AC_data['TF'][:, J, K]

Fit the tranfer functions using the vector fitting algorithm and create the `lsim` objects:

In [ ]:
systems, _ = fit_TF(frequencies, TF)

Define the inputs:

In [ ]:
dt = tran_config['dt']
tend = tran_config['tstop']
time = np.r_[0 : tend + dt / 2 : dt]
N_samples = time.size
U = np.tile(
    np.array([u[0] for u in input_fun(time)]),
    (N_inputs, 1)
)

Filter the inputs using the `lsim` objects above:

In [ ]:
Y = filter_inputs(systems, U, time)

Plot the results:

In [ ]:
cmap = plt.get_cmap('tab20')
k = 0
fig, ax = plt.subplots(len(devices_to_consider), max_N_vars, sharex=True)
t0 = tran_time[-1] - 10 / F
for i, dev in enumerate(devices_to_consider):
    var_name = var_names_dict[dev]
    X = tran_data[dev][var_name].T
    for j in range(max_N_vars):
        idx = tran_time > t0
        ax[i, j].plot(tran_time[idx], X[j, idx] - 1, color=cmap(k*2+1), lw=3, label='PF')
        idx = time > t0
        ax[i, j].plot(time[idx], Y[k, idx], '--', color=cmap(k*2), lw=1, label='ss')
        tokens = full_var_names[k].split('-')[-1].split('.')
        lbl = '{} {}'.format(tokens[0], tokens[2])
        ax[i, j].set_ylabel(lbl)
        k += 1
ax[0, 1].legend(loc='upper right', frameon=True, fontsize=7)
for a in ax[-1,:]:
    a.set_xlabel('Time (s)')
ax[0, 0].set_xlim([time[-1] - 10 / F, time[-1]])
sns.despine()
fig.tight_layout()
plt.savefig(FIGURES_DIR / f'Sardinia_GFM_pref_{F}_Hz_PF_vs_ss.pdf')